In [ ]:

import numpy as np
import torch 


In [235]:
colors = ['B', 'G', 'R', 'Y']
words = ['B', 'G', 'R', 'Y']

# all possible combinations of colors and words with 2 elements
# e.g. ['B', 'G']


def get_all_combinations():
    all_combinations = []
    for i in range(4):
        for j in range(4):
            all_combinations.append([colors[i], words[j]])
    return all_combinations

In [236]:
X = get_all_combinations()

In [237]:
X_old = get_all_combinations()

In [238]:
X

[['B', 'B'],
 ['B', 'G'],
 ['B', 'R'],
 ['B', 'Y'],
 ['G', 'B'],
 ['G', 'G'],
 ['G', 'R'],
 ['G', 'Y'],
 ['R', 'B'],
 ['R', 'G'],
 ['R', 'R'],
 ['R', 'Y'],
 ['Y', 'B'],
 ['Y', 'G'],
 ['Y', 'R'],
 ['Y', 'Y']]

In [239]:
y = [_[0] for _ in X]

In [240]:
y

['B',
 'B',
 'B',
 'B',
 'G',
 'G',
 'G',
 'G',
 'R',
 'R',
 'R',
 'R',
 'Y',
 'Y',
 'Y',
 'Y']

In [241]:
# transform X into one hot: for each element in X, we have a vector of 8 elements 
# where the index of the color is 1 and the index of the word is 1

def transform_X(X):
    X_transformed = []
    for i in range(len(X)):
        X_transformed.append([0]*8)
        X_transformed[i][colors.index(X[i][0])] = 1
        X_transformed[i][words.index(X[i][1])+4] = 1
    return X_transformed

In [242]:
X = transform_X(X)

In [243]:
# one hot encoding of y with 4 elements per element in y
def transform_y(y):
    y_transformed = []
    for i in range(len(y)):
        y_transformed.append([0]*4)
        y_transformed[i][colors.index(y[i])] = 1
    return y_transformed

y = transform_y(y)

In [244]:
# train on 80% of the data
X_train = X[:int(len(X)*0.8)]
y_train = y[:int(len(y)*0.8)]

# test on 20% of the data
X_test = X[int(len(X)*0.8):]
y_test = y[int(len(y)*0.8):]

In [245]:
y_test

[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1]]

In [246]:
# neural network with 2 hidden layers of 10 neurons each predicting the color from X

import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8, 8)
        self.fc2 = nn.Linear(8, 8)
        self.fc3 = nn.Linear(8, 4)
        
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

# loss function and optimizer
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.05, momentum=0.9)

# train the network
for epoch in range(1000):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(len(X_train)):
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(torch.tensor(X_train[i]).float())
        loss = criterion(outputs, torch.tensor(y_train[i]).float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    if epoch % 100 == 99:    # print every 100 epochs
        print('[%d] loss: %.3f' %
              (epoch + 1, running_loss / len(X_train)))
        running_loss = 0.0

print('Finished Training')

[100] loss: 0.000
[200] loss: 0.000
[300] loss: 0.000
[400] loss: 0.000
[500] loss: 0.000
[600] loss: 0.000
[700] loss: 0.000
[800] loss: 0.000
[900] loss: 0.000
[1000] loss: 0.000
Finished Training


In [253]:
# test the network
correct = 0
total = 0

with torch.no_grad():
    for i in range(len(X_train)):
        outputs = net(torch.tensor(X_train[i]).float())
        _, predicted = torch.max(outputs.data, 0)
        _, actual = torch.max(torch.tensor(y_train[i]).float(), 0)
        total += 1
        correct += (predicted == actual).sum().item()

print('Accuracy of the network on the test data: %d %%' % (
    100 * correct / total))


Accuracy of the network on the test data: 100 %


In [252]:
total

4

In [248]:
# transform every element in activations to 8 x 2 matrices instead of 1 x 8 vectors

def transform_activations(activations):
    for i in range(len(activations)):
        activations[i][0] = np.reshape(activations[i][0], (8, 1))
        activations[i][1] = np.reshape(activations[i][1], (8, 1))
        # combine the two matrices into one
        activations[i] = np.concatenate((activations[i][0], activations[i][1]), axis=1).T
    return activations

In [249]:
# plot every activation as a 8 x 2 heatmap

def plot_activations(activations):
    for i in range(len(activations)):
        plt.imshow(activations[i], cmap='seismic', interpolation='nearest')
        plt.title(X_old[i])
        # add heatmap legend
        plt.colorbar()
        plt.show()

In [250]:
plot_activations(transform_activations(activations))

ValueError: could not broadcast input array from shape (8,1) into shape (8,)